In [1]:
import reaction_parallel_cython
import numpy as np

In [2]:
reaction_parallel_cython.vector_to_numpy_2d()

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19],
       [20, 21, 22, 23],
       [24, 25, 26, 27],
       [28, 29, 30, 31],
       [32, 33, 34, 35],
       [36, 37, 38, 39]], dtype=int32)

In [3]:
N = 20

cellsize = 100

Particle_dtype = np.dtype([
    ('pos', np.float64, (3,)),
    ('vel', np.float64, (3,)),
    ('E', np.float64),
    ('id', np.int32)
], align=True)  # 添加 align=True

Cell_dtype = np.dtype([
    ('id', np.int32),
    ('index', np.int32, (3,)),
    ('film', np.int32, (5,)),
    ('normal', np.float64, (3,))
], align=True)  # 添加 align=True

particle = np.zeros(N, dtype=Particle_dtype)
# cell = np.zeros((cellsize, cellsize, cellsize), dtype=CELLDTYPE)

pos = np.random.rand(N, 3)*99
vel = np.random.rand(N, 3)
energy = np.linalg.norm(vel, axis=1)
vel[:,0] = np.divide(vel[:,0], energy)
vel[:,1] = np.divide(vel[:,1], energy)
vel[:,2] = np.divide(vel[:,2], energy)


# 填充粒子数据
particle['pos'] = pos
particle['vel'] = vel
particle['E'] = 100
particle['id'] = 0
particle[int(N/2):]['id'] = 1
cell = np.zeros((cellsize, cellsize, cellsize), dtype=Cell_dtype)

nn = np.array([0, 0, 1])
# nn = nn/np.linalg.norm(nn)

cell['id'] = 1
cell['index'] = np.array([1,2,3])
cell['film'] = np.array([10,10,10,10,0])
cell['normal'] = nn

cell[:,:,50:]['id'] = 1
print(Particle_dtype.itemsize)
print(Cell_dtype.itemsize)

64
64


In [4]:
particle

array([([67.42350719, 66.32756759, 29.76493221], [0.71515727, 0.6187403 , 0.32513153], 100., 0),
       ([86.61170074, 27.17961205, 34.16047831], [0.84551936, 0.04171888, 0.53231245], 100., 0),
       ([38.09668414, 26.29813796, 14.76049037], [0.01034896, 0.68602543, 0.72750396], 100., 0),
       ([96.83959164, 56.1455553 , 39.98184162], [0.51714793, 0.54217267, 0.66227397], 100., 0),
       ([98.88133776, 18.15061335, 58.17243959], [0.15721847, 0.55472005, 0.81704836], 100., 0),
       ([85.24434138,  6.16153084, 33.7731614 ], [0.98548499, 0.1670424 , 0.03026849], 100., 0),
       ([21.47600809, 46.48710595, 62.04900151], [0.56462151, 0.55173306, 0.61383482], 100., 0),
       ([13.39807411, 50.3929777 , 89.16098479], [0.20660403, 0.52496979, 0.82566427], 100., 0),
       ([15.22210401,  8.50197971, 90.66569645], [0.6267246 , 0.34161888, 0.7003662 ], 100., 0),
       ([ 2.55448345, 18.25891498, 16.19873631], [0.27316473, 0.1878323 , 0.94345114], 100., 0),
       ([10.16061693, 83.95538

In [5]:
a = reaction_parallel_cython.particle_parallel_vector(particle, cell)

In [6]:
# print(a.shape)
print(a[0])
print(a[1])
print(a[2])
print(a[3])

[0 0 0 0 0 0 0 0 0 0 3 1 2 0 1 2 3 2 0 0]
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 1 1 1 0]
 [1 1 1 1 0]
 [1 1 1 1 0]
 [1 0 0 0 0]
 [1 1 1 1 0]
 [1 1 1 1 0]
 [1 1 0 1 0]
 [1 1 1 1 0]
 [1 1 1 1 0]
 [1 1 1 1 0]]
[[0.79654298 0.         0.         0.         0.        ]
 [0.49686129 0.         0.         0.         0.        ]
 [0.43254171 0.         0.         0.         0.        ]
 [0.30453644 0.         0.         0.         0.        ]
 [0.25804682 0.         0.         0.         0.        ]
 [0.8517575  0.         0.         0.         0.        ]
 [0.42648376 0.         0.         0.         0.        ]
 [0.47373716 0.         0.         0.         0.        ]
 [0.12917458 0.         0.         0.         0.        ]
 [0.32711078 0.         0.         0.         0.        ]
 [0.82337419 0.14625347 0.42068436 0.82784474 0.        ]
 [0.67875499 0.84626541 0.0339859  0.77433121 0.        ]
 

In [7]:
print(a[4])

[[-1  1  0  0  0]
 [-1  1  0  0  0]
 [-1  1  0  0  0]
 [-1  1  0  0  0]
 [-1  1  0  0  0]
 [-1  1  0  0  0]
 [-1  1  0  0  0]
 [-1  1  0  0  0]
 [-1  1  0  0  0]
 [-1  1  0  0  0]
 [ 0  0  0 -1  0]
 [ 0 -1  0  0  0]
 [ 0  0 -1  0  0]
 [-1  0  0  0  0]
 [ 0 -1  0  0  0]
 [ 0  0 -1  0  0]
 [ 0  0  0 -1  0]
 [ 0  0 -1  0  0]
 [-1  0  0  0  0]
 [-1  0  0  0  0]]


In [ ]:
a = reaction_parallel_cython.react_add_test2(particle[0])

print(a)

In [ ]:
a = reaction_parallel_cython.react_add_test3(particle)

print(a)

In [3]:
a = reaction_parallel_cython.particle_parallel(particle, cell)

In [ ]:
a.min()

In [ ]:
print(a.shape)
print(a[:200])

In [ ]:
%timeit reaction_parallel_cython.particle_parallel(particle, cell)

In [ ]:
print(a)

In [50]:
@jit(nopython=True, parallel=True)
def particle_parallel_numba(particles, cell):

    dot_product = np.zeros(particles.shape[0], dtype=np.double)

    for i in prange(particles.shape[0]):
        cellijk = particles[i]['pos'].astype(np.int32)

        if cell[cellijk[0],cellijk[1], cellijk[2]]['id'] == 1:
            dot_product[i] = np.dot(particles[i]['vel'], cell[cellijk[0],cellijk[1], cellijk[2]]['normal'])

    return dot_product

In [ ]:
np.arange(0, np.pi/2, 0.01, dtype=np.double).shape

In [ ]:
np.arange(0, np.pi/2, 0.01, dtype=np.double)

In [ ]:
np.linspace(0, np.pi/2, 180, dtype=np.double)

In [ ]:
np.linspace(0, np.pi/2, 180, dtype=np.double).shape

In [38]:

def particle_parallel(particles, cell):

    dot_product = np.zeros(particles.shape[0], dtype=np.double)

    for i in range(particles.shape[0]):
        celli = particles[i]['pos'][0].astype(np.int32)
        cellj = particles[i]['pos'][1].astype(np.int32)
        cellk = particles[i]['pos'][2].astype(np.int32)

        if cell[celli, cellj, cellk]['id'] == 1:
            dot_product[i] = np.dot(particles[i]['vel'], cell[celli, cellj, cellk]['normal'])

    return dot_product

In [ ]:
%timeit particle_parallel_numba(particle, cell)

In [ ]:
%timeit particle_parallel(particle, cell)